In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os 

import sys; sys.path.append('/home/mitch/school/mfl/')
import src.json_utils

In [127]:
raw = '/home/mitch/Dropbox/data/mexico_fls/raw/2005/'
interim = '/home/mitch/Dropbox/data/mexico_fls/interim2/2005/'

figs = '/home/mitch/school/mfl/fig/'
dicts= '/home/mitch/school/mfl/dicts/'

In [128]:
rename = src.json_utils.load_json(dicts + 'rename_ind_2005.json')

In [129]:
os.chdir(dicts)
education = src.json_utils.load_json('education.json')

In [130]:
ids = ['folio', 'ls', 'pid_link']
keep = ['folio', 'ls', 'pid_link']
os.chdir(raw + 'book IIIB/data/')
portad = pd.read_stata('iiib_portad.dta')
portad = portad.sort_values(by=['folio', 'ls'])

In [131]:
portad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20607 entries, 0 to 20606
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   catorcena  20607 non-null  float32
 1   edad       20606 non-null  float32
 2   edo_civil  20607 non-null  int8   
 3   ent        20607 non-null  float32
 4   folio      20607 non-null  object 
 5   ls         20607 non-null  object 
 6   pid_link   20607 non-null  object 
 7   rel        20607 non-null  float32
dtypes: float32(4), int8(1), object(3)
memory usage: 986.1+ KB


In [132]:
os.chdir(raw + 'book IIIA/data/')
weights_3a = pd.read_stata('weights.dta')
weights_3a = (weights_3a.query('fac_3a>0')
              .drop_duplicates()
              .sort_values(by=['folio', 'ls']))
weights_3a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19544 entries, 0 to 35087
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   folio   19544 non-null  object
 1   ls      19544 non-null  object
 2   fac_3a  19544 non-null  int32 
dtypes: int32(1), object(2)
memory usage: 534.4+ KB


In [133]:
# ids
os.chdir(raw + 'book C/data/')
df_ls = pd.read_stata('c_ls.dta')
df_ls = df_ls.drop(columns='pid_link')

os.chdir(raw + 'book IIIA/data/')
# employment
df_tb = pd.read_stata('iiia_tb.dta')
df_tb = df_tb.drop(columns='pid_link')

# education
df_ed = pd.read_stata('iiia_ed.dta')
df_ed = df_ed.drop(columns='pid_link')

# non labor income, is ANNUAL
df_iin = pd.read_stata('iiia_iin.dta')
df_iin = df_iin.drop(columns='pid_link')

os.chdir(raw + 'book IIIB/data/')
# credit
df_cr = pd.read_stata('iiib_cr.dta')
df_cr = df_cr.drop(columns='pid_link')

# credit 1
df_cr1 = pd.read_stata('iiib_cr1.dta')
df_cr1 = df_cr1.drop(columns='pid_link')

# family structure (gender)
df_thi = pd.read_stata('iiib_thi.dta')
df_thi = df_thi.drop(columns='pid_link')

In [134]:
df = (weights_3a.merge(portad, on=['folio', 'ls'], how='left')
        .merge(df_ls, on=['folio', 'ls'], how='left')
        .merge(df_tb, on=['folio', 'ls'], how='left')
        .merge(df_ed, on=['folio', 'ls'], how='left')
        .merge(df_cr, on=['folio', 'ls'], how='left')
        .merge(df_cr1, on=['folio', 'ls'], how='left')
        .merge(df_thi, on=['folio', 'ls'], how='left')
        )

In [135]:
df = df.rename(columns=rename)
df = df[rename.values()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19930 entries, 0 to 19929
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hhid                    19930 non-null  object 
 1   hhmember                19930 non-null  object 
 2   weight_ind              19930 non-null  int32  
 3   age                     19463 non-null  float32
 4   state                   19464 non-null  float32
 5   education               17500 non-null  float32
 6   gender                  19452 non-null  float32
 7   has_income              6365 non-null   float64
 8   income                  5048 non-null   float32
 9   has_income_secondjob    112 non-null    float32
 10  income_secondjob        97 non-null     float32
 11  has_wage                19590 non-null  object 
 12  wage                    841 non-null    float32
 13  has_piecework           19590 non-null  object 
 14  piecework               831 non-null  

In [136]:
df['hh_size'] = df.groupby('hhid')['hhmember'].transform('size')
df['education'] = df['education'].astype(str).replace(education)
df['gender'] = df['gender'].replace({1.0:1, 2.0:0, 3.0:0})

In [ ]:
has_values = {
    'wage':'A',
    'piecework':'B',
    'tips':'C',
    'extrahours':'D',
    'christmasbonus':'E',
    'bonus':'F',
    'profitdist':'H',
    'other':'M'
}

In [ ]:
dk = 8.0
df['total_income'] = 0.0

total = df['has_income'] == 3.0 
detailed = df['has_income'] == 1.0 
dk = df['has_income'] == 8.0 

df.loc[total, 'total_income'] = df.loc[total, 'income']

for income in ['wage', 'piecework', 'tips', 'extrahours',
               'christmasbonus', 'bonus', 'profitdist', 'other']:
    knows = df['has_' + income] == has_values[income]
    dk = df['has_' + income] == 8

    df.loc[dk, income] = 0.00
    df[income] = df[income].fillna(0.00)

In [ ]:
income_sources = [
                  'income',
                  'wage',
                  'piecework',
                  'tips',
                  'extrahours',
                  'other',
                  'christmasbonus',
                  'bonus',
                  'profitdistribution',
                  'income_second_job'
                  ]
                  #'netIncomeMainBusiness', 'netIncomeSecondBusiness']

df['total_income'] = df[income_sources].sum(axis=1)

# Debts

In [ ]:
# more formal measure of debts
df['asked_for_loan_12mth'] = df['cr10'] == 1.0
df['not_asked_for_loan_32mth'] = df['cr10'] == 1.0

loan_locations = {
    1.0:'bank',
    2.0:'savings_fund',
    3.0:'moneylender',
    4.0:'relative',
    5.0:'friends',
    6.0:'work',
    7.0:'pawnshop',
    8.0:'verbal_agreement_credit_program',
    9.0:'other_govt',
    10.0:'other',
    -1.0: np.nan
}
formal = ['bank', 'savings_fund', 'moneylender', 'pawnshop']
informal = ['relative', 'friends', 'work', 'verbal_agreement_credit_program', 'other_govt', 'other']
friends_relatives = ['relative', 'friends']

df.loc[np.isnan(df['where_get_loan']), 'where_get_loan'] = -1.0


In [ ]:
df['where_get_loan'] = df['where_get_loan'].replace(loan_locations)
df['has_loan_formal']    = df['where_get_loan'].isin(formal)
df['has_loan_informal']  = df['where_get_loan'].isin(informal)
df['has_loan_friends_relatives']  = df['where_get_loan'].isin(friends_relatives)
df['has_loan'] = df['has_loan_formal'] | df['has_loan_informal'] | df['has_loan_friends_relatives']

got_loan = df['received_loan'] == 1.0

df[~got_loan, 'loan_size'] = 0.0
#df.loc[got_loan, 'loan_size'] = df.loc[got_loan, 'cr19']

df['loan_formal']   = df['loan_size'] * (df['has_loan_formal'] + 0.0)
df['loan_informal'] = df['loan_size'] * (df['has_loan_informal'] + 0.0)
df['loan_friends_relatives']   = df['loan_size'] * (df['has_loan_friends_relatives'] + 0.0)


# Savings

In [149]:
df.has_savings.value_counts()

3.0    17042
1.0     2156
7.0      127
Name: has_savings, dtype: int64

In [ ]:
df['has_savings'] = df['cr27'] == 1.0
df['no_savings'] = df['cr27'] == 3.0
dk_savings = df['cr27'] == 8.0

df['savings'] = np.nan
df.loc[df.has_savings, 'savings'] = df.loc[df.has_savings, 'cr28']
df.loc[df.no_savings, 'savings'] = 0.0

savings_no_savings     = df['cr29_1a'] == 1.0
savings_bank           = df['cr29_1b'] == 2.0
savings_coop           = df['cr29_1c'] == 3.0
savings_savings_bank   = df['cr29_1d'] == 4.0
savings_friend_not_hhm = df['cr29_1e'] == 5.0
savings_afores         = df['cr29_1f'] == 6.0
savings_caja_solidaria = df['cr29_1g'] == 7.0
savings_house          = df['cr29_1h'] == 8.0
savings_work           = df['cr29_1i'] == 9.0
savings_other          = df['cr29_1j'] == 10.0

df['has_savings_formal']   = savings_bank | savings_coop | savings_savings_bank | savings_caja_solidaria
df['has_savings_informal'] = savings_friend_not_hhm | savings_house | savings_work 


df['savings_formal']   = (df['has_savings_formal'] + 0.0) * df['cr28']
df['savings_informal'] = (df['has_savings_informal'] + 0.0) * df['cr28']

df.loc[df.no_savings, 'savings_formal'] = 0
df.loc[df.no_savings, 'savings_informal'] = 0

both = df['has_savings_formal'] & df['has_savings_informal']

df.loc[both, 'savings_formal']   = np.nan
df.loc[both, 'savings_informal'] = np.nan

df['has_liquid_formal'] = df['has_savings_formal'] | df['loan_formal']
df['liquid_formal'] = df['savings_formal'] - df['loan_formal']

df['has_liquid'] = df['has_savings'] | df['has_loan']

In [ ]:
df = (df.query('age >= 20')
      .query('age <= 65')
)

In [150]:
df['hh_total_income'] = df.groupby('hhid')['total_income'].transform('sum')
df['hh_savings'] = df.groupby('hhid')['savings'].transform('sum')
df['hh_debts'] = df.groupby('hhid')['debts'].transform('sum')

df['hh_has_savings'] = df.groupby('hhid')['has_savings'].transform(lambda x : x.size() > 0)

In [ ]:
df['is_hhm'] = df['ls05_1'] 
df = df[df['is_hhm'] == 1.0]

os.chdir(interim)
df[keep].to_csv('ind_2005.csv', index=False)

# Savings

In [116]:
import importlib
importlib.reload(utils)
df = utils.sum_over_household(df, 'savings')
df = utils.sum_over_household(df, 'savings_formal')
df = utils.sum_over_household(df, 'savings_informal')
df = utils.sum_over_household(df, 'annual_labor_income')
df = utils.sum_over_household(df, 'annual_wage')
df = utils.sum_over_household(df, 'annual_piecework')
df = utils.sum_over_household(df, 'annual_tips')
df = utils.sum_over_household(df, 'annual_extrahours')
df = utils.sum_over_household(df, 'annual_other')
df = utils.sum_over_household(df, 'annual_christmasbonus')
df = utils.sum_over_household(df, 'annual_bonus')
df = utils.sum_over_household(df, 'annual_profitdistribution')
df = utils.sum_over_household(df, 'annual_income_second_job')
df = utils.sum_over_household(df, 'netIncomeMainBusiness')
df = utils.sum_over_household(df, 'netIncomeSecondBusiness')
df = utils.sum_over_household(df, 'loan_formal')
df = utils.sum_over_household(df, 'loan_informal')
df = utils.sum_over_household(df, 'loan_friends_relatives')
df = utils.sum_over_household(df, 'liquid_formal')


df = utils.any_in_household(df, 'has_liquid')
df = utils.any_in_household(df, 'has_savings')
df = utils.any_in_household(df, 'has_savings_formal')
df = utils.any_in_household(df, 'has_savings_informal')
df = utils.any_in_household(df, 'has_loan_formal')
df = utils.any_in_household(df, 'has_loan_informal')
df = utils.any_in_household(df, 'has_loan_friends_relatives')
df = utils.any_in_household(df, 'has_loan')
df = utils.any_in_household(df, 'has_liquid_formal')
df = utils.any_in_household(df, 'asked_for_loan_12mth')


In [117]:
keep += ['has_liquid_hh', 'savings_hh', 'savings_formal_hh', 'savings_informal_hh']
keep += ['has_savings_hh', 'has_savings_formal_hh', 'has_savings_informal_hh']
keep += ['loan_formal_hh', 'loan_informal_hh', 'loan_friends_relatives_hh']
keep += ['has_loan_hh', 'has_loan_formal_hh', 'has_loan_informal_hh', 'has_loan_friends_relatives_hh']
keep += ['liquid_formal_hh', 'has_liquid_formal_hh']
keep += ['annual_labor_income_hh', 'netIncomeMainBusiness_hh', 'netIncomeSecondBusiness_hh']

In [118]:
inner = df[['folio', 'annual_labor_income']].groupby('folio')['annual_labor_income'].apply(np.sum).rename('annual_labor_income_hh_inner')
outer = df[['folio', 'annual_labor_income']].groupby('folio')['annual_labor_income'].apply(np.sum).rename('annual_labor_income_hh_outer')
df_inner = df.copy().merge(inner, on='folio', how='inner')
df_outer = df.copy().merge(outer, on='folio', how='outer')

# loans regardless of location

saved
